In [ ]:
from simulation import Simulation
from particle import Particle, Particles
from edge import ExcitatorSin, MirrorEdge

import numpy as np

In [ ]:
mypath = os.path.dirname(os.path.abspath(__file__))
mypath = "C:\\Users\\leog\\Desktop\\lg2021stage\\output"

In [ ]:
duration = 2.0 # duration of simulation [s]
length = 1.0 # [m]
tension = 1.0 # [N]
density = 0.01 # [kg/m]
space_steps = 256 # string discretisation

celerity = np.sqrt(tension/density)

dx = length/space_steps
dt = dx/celerity
time_steps = int(duration/dt)

In [ ]:
initial_field = [0.0]*space_steps
initial_vel_field = [0.0]*space_steps

In [ ]:
signal_pulsation = 10.0*2.0*np.pi*signal_freq # [rad/s]
edge_left = ExcitatorSin(dt, 0.01, signal_pulsation, 0.0)
edge_right = MirrorEdge()

In [ ]:
particle_a = Particle(100, 0.0, 0.01, 2*np.pi, True, space_steps)
particle_b = Particle(200, 0.0, 0.02, 2*np.pi, True, space_steps)
particles = Particles(particle_a, particle_b)

In [ ]:
simu = Simulation(dt, time_stemps, space_steps, length, density, tension, edge_left, edge_right, initial_field, initial_vel_field, particles)
print(simu)

In [ ]:
simu.run(path, anim=True, file=True, compress=False)